In [6]:
import json
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from sqlalchemy import URL
import numpy as np 
import plotly.graph_objects as go

import os

In [7]:
path = os.getcwd()

# Загрузка датафреймов

In [8]:
data = pd.read_csv(f"{path}\\Данные\\Исходные\\Переменные\\Отн_влажность.csv", encoding = "windows-1251")

# Относительная влажность

In [9]:
data

,date_loc,value,site_name,site_code,lon,lat,value_id,name_rus
0,1972-11-21 00:00:00.000,61.0,Анучино,31981,133.066667,43.966667,1004,"Относ. влаж., сут ср"
1,1972-11-22 00:00:00.000,65.0,Анучино,31981,133.066667,43.966667,1004,"Относ. влаж., сут ср"
2,1972-11-23 00:00:00.000,74.0,Анучино,31981,133.066667,43.966667,1004,"Относ. влаж., сут ср"
3,1972-11-24 00:00:00.000,79.0,Анучино,31981,133.066667,43.966667,1004,"Относ. влаж., сут ср"
4,1972-11-25 00:00:00.000,79.0,Анучино,31981,133.066667,43.966667,1004,"Относ. влаж., сут ср"
...,...,...,...,...,...,...,...,...
20929995,1974-09-15 00:00:00.000,83.0,Средняя Олекма,30484,120.550000,55.430000,1004,"Относ. влаж., сут ср"
20929996,1974-09-16 00:00:00.000,50.0,Средняя Олекма,30484,120.550000,55.430000,1004,"Относ. влаж., сут ср"
20929997,1974-09-17 00:00:00.000,79.0,Средняя Олекма,30484,120.550000,55.430000,1004,"Относ. влаж., сут ср"
20929998,1974-09-18 00:00:00.000,79.0,Средняя Олекма,30484,120.550000,55.430000,1004,"Относ. влаж., сут ср"


# Уникальные site_id для параметра температуры 

In [10]:
print("Количество уникальных site_id:" , len(pd.unique(data["site_code"])))

Количество уникальных site_id: 1250


In [11]:
list(pd.unique(data["site_code"]))

[31981,
 31986,
 31987,
 31989,
 32010,
 32014,
 32017,
 32027,
 32036,
 32053,
 32054,
 32059,
 32061,
 32069,
 32071,
 32076,
 32077,
 32088,
 32092,
 32098,
 32116,
 32121,
 32133,
 32136,
 32143,
 32145,
 32149,
 32150,
 32156,
 32158,
 32165,
 32166,
 32174,
 32213,
 32215,
 32222,
 32246,
 32287,
 32363,
 32379,
 32389,
 32408,
 32411,
 32447,
 32477,
 32509,
 32539,
 32547,
 32562,
 32583,
 32586,
 32594,
 32618,
 33166,
 34001,
 34003,
 34005,
 34009,
 34013,
 34036,
 34047,
 34056,
 34059,
 34063,
 34069,
 34072,
 34083,
 34098,
 34109,
 34110,
 34112,
 34116,
 34121,
 34123,
 34139,
 34146,
 34152,
 34163,
 34169,
 34171,
 34173,
 34186,
 34199,
 34202,
 34213,
 34214,
 34237,
 34238,
 34240,
 34247,
 34253,
 34254,
 34262,
 34267,
 34273,
 34289,
 34321,
 34336,
 34344,
 34352,
 34356,
 34357,
 34362,
 34363,
 34373,
 34391,
 34432,
 34438,
 34445,
 34461,
 34476,
 34535,
 34539,
 34545,
 34552,
 34555,
 34561,
 34578,
 34579,
 34625,
 34635,
 34636,
 34644,
 34646,
 34655,


# Минимальная и максимальная дата для каждого из сайтов

In [12]:
data_for_date = data[["date_loc", "site_code", "site_name"]]

max_data_for_date = data_for_date.groupby(["site_code", "site_name"]).max()
max_data_for_date.rename(columns={"date_loc": 'date_loc_max'}, inplace=True)
min_data_for_date = data_for_date.groupby(["site_code", "site_name"]).min()
min_data_for_date.rename(columns={"date_loc": 'date_loc_min'}, inplace=True)


In [13]:
date_stat = pd.concat([min_data_for_date, max_data_for_date], axis=1).reset_index()

# Среднее, минимальное и максимальное значение value для site_id и количество измерений для кадой переменной

In [14]:
data_for_stats = data[["value", "site_code", 'site_name',]]

data_for_loc = data[["site_code", "lon", "lat"]]
data_for_loc = data_for_loc.groupby(["site_code"]).mean()
# среднее
mean_by_site = data_for_stats.groupby(["site_code", "site_name"]).mean()
mean_by_site.rename(columns={"value": 'value_mean'}, inplace=True)
# максимальное
max_by_site = data_for_stats.groupby(["site_code", "site_name"]).max()
max_by_site.rename(columns={"value": 'value_max'}, inplace=True)
# минимальное
min_by_site = data_for_stats.groupby(["site_code", "site_name"]).min()
min_by_site.rename(columns={"value": 'value_min'}, inplace=True)
# количество 
amount_by_site = data_for_stats.groupby(["site_code", "site_name"]).count()
amount_by_site.rename(columns={"value": 'value_amount'}, inplace=True)

# Мерджим все дата фреймы для общей таблицы

In [15]:

value_stat = pd.concat([mean_by_site, min_by_site, max_by_site, amount_by_site], axis=1).reset_index()


In [16]:
stat = pd.concat([value_stat.set_index(["site_code", "site_name"]), date_stat.set_index(["site_code", "site_name"])], axis=1).reset_index()

In [17]:
stat

,site_code,site_name,value_mean,value_min,value_max,value_amount,date_loc_min,date_loc_max
0,20107,Баренцбург,78.822597,39.0,100.0,15462,1966-01-01 00:00:00.000,2014-08-31 00:00:00.000
1,20674,остров Диксон,86.785698,47.0,100.0,17424,1966-01-01 00:00:00.000,2014-08-31 00:00:00.000
2,20891,Хатанга,77.710942,30.0,100.0,17775,1966-01-01 00:00:00.000,2014-08-31 00:00:00.000
3,20946,им.Е.К.Федорова,86.040214,51.0,100.0,14920,1967-01-01 00:00:00.000,2014-07-31 00:00:00.000
4,20967,Сеяха,85.369103,49.0,100.0,13503,1977-01-01 00:00:00.000,2014-08-31 00:00:00.000
...,...,...,...,...,...,...,...,...
1245,54273,Хуадиан,71.068338,23.0,100.0,21057,1956-01-01 00:00:00.000,2013-09-30 00:00:00.000
1246,54284,"East hills,",67.198390,23.0,100.0,20616,1956-12-01 00:00:00.000,2013-09-30 00:00:00.000
1247,54285,songjiang,71.650243,25.0,99.0,20377,1957-11-01 00:00:00.000,2013-09-30 00:00:00.000
1248,54287,tianchi,77.163602,9.0,100.0,13759,1958-10-01 00:00:00.000,2013-09-30 00:00:00.000


In [18]:
stat.to_csv("full_stat_humidity.csv", sep=";", encoding="cp1251")

In [19]:
amount_by_site = amount_by_site.reset_index().set_index("site_code")
for_mapping = pd.concat([amount_by_site, data_for_loc], axis=1).reset_index()

In [20]:
for_mapping

,site_code,site_name,value_amount,lon,lat
0,20107,Баренцбург,15462,NaN,NaN
1,20674,остров Диксон,17424,80.400003,73.513611
2,20891,Хатанга,17775,102.470000,71.980000
3,20946,им.Е.К.Федорова,14920,59.091111,70.446669
4,20967,Сеяха,13503,72.513611,70.170003
...,...,...,...,...,...
1245,54273,Хуадиан,21057,126.750000,42.983333
1246,54284,"East hills,",20616,127.566667,42.100000
1247,54285,songjiang,20377,128.250000,42.533333
1248,54287,tianchi,13759,128.083333,42.016667


In [21]:
# новый столбец для создания названия на карте 
for_mapping['name_map'] = for_mapping['site_name'].astype(str) + ' (' + for_mapping['site_code'].astype(str) + ') кол-во измерений:' + for_mapping['value_amount'].astype(str)

stat.to_csv("map_humid.csv", sep=";", encoding="cp1251")

# Создаем карту

In [22]:

# задаем размеры
layout = go.Layout(
    autosize=False,
    width=1250,
    height=1000,
    xaxis=go.layout.XAxis(linecolor="black", linewidth=1, mirror=True),
    yaxis=go.layout.YAxis(linecolor="black", linewidth=1, mirror=True),
    margin=go.layout.Margin(l=50, r=50, b=100, t=100, pad=4),
)

# структура карты 
fig = go.Figure(go.Scattermapbox(lat=for_mapping["lat"],
                                 lon=for_mapping["lon"], 
                                text= for_mapping["name_map"],
                                marker=dict(colorbar=dict(title="Количество измерений"),
                                             color=for_mapping['value_amount'],
                                             )
                                 ), 
                 layout=layout)
map_center = go.layout.mapbox.Center(lat=58.38, 
                                     lon=97.45)
fig.update_layout(mapbox_style="open-street-map",
                  mapbox=dict(center=map_center, zoom=2))
fig.show()

In [30]:
lol = pd.read_csv(f"{os.getcwd()}\\streamlit\\full_stat_максимальная температура.csv", encoding = "windows-1251", sep = ";")

In [25]:
os.getcwd()

'd:\\geogr_api'